#### Preparing

In [1]:
import pandas as pd

test_df = pd.read_parquet("./data/66-test-dataset-with-adrd.parquet")

test_df.head()

,Question,Context,Answer,adrd_qwen3_14b,adrd_dpv3,gpt-o3,gpt-4.1,adrd_qwen3_30b_A3B
0,Can you recommend activities that are suitable...,Content: What are some tips for Finding Activi...,When looking for activities for someone with d...,Engaging a loved one with dementia in meaningf...,I understand how important it is to find meani...,I understand that finding the right activities...,Thank you for reaching out with this important...,I understand how challenging it can be to find...
1,What are the benefits of hospice care for indi...,Content: How to be an effective caregiver in t...,Hospice care offers several benefits for indiv...,Caring for someone with advanced Alzheimer's i...,Hospice care can provide immense comfort and s...,I understand that navigating the complexities ...,Caring for a loved one with advanced Alzheimer...,I'm deeply sorry you're navigating this challe...
2,How can I navigate difficult family dynamics w...,Content: How to Handle Seemingly Manipulative ...,When navigating difficult family dynamics whil...,Navigating family dynamics while caring for a ...,Navigating difficult family dynamics while car...,I understand how overwhelming it can feel to j...,Caring for a loved one with Alzheimer's can be...,I'm so sorry you're navigating such a challeng...
3,What are the early signs of dementia?,Content: None The early warning signs of demen...,The early signs of dementia can vary from pers...,It's completely understandable to feel concern...,I understand how important it is to recognize ...,I understand how overwhelming it can be to not...,Thank you for reaching out with this important...,I'm so sorry you're navigating this journey—it...
4,"How does palliative care differ from hospice, ...",Content: What end-of-life care options are ava...,Palliative care and hospice care are both form...,Understanding the distinction between palliati...,Palliative care and hospice care both aim to i...,I understand that navigating the differences b...,Thank you for reaching out with this important...,I'm deeply sorry for the challenges you're fac...


In [2]:
question_array = test_df.Question.to_list()

print(question_array)

['Can you recommend activities that are suitable for someone with dementia to engage in and enjoy?', "What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease?", "How can I navigate difficult family dynamics when caring for a loved one with Alzheimer's?", 'What are the early signs of dementia?', 'How does palliative care differ from hospice, and how can it help improve the quality of life for someone with advanced dementia?', "How can I ensure my loved one's safety at home, especially if they have a tendency to wander or become agitated?", 'What activities are suitable for people with dementia, and how can I encourage participation?', 'Are there any risk factors that can contribute to developing delirium?', 'Are certain types of anesthesia more likely to cause delirium?', "How do I navigate legal issues related to caring for a loved one with Alzheimer's, such as power of attorney and guardianship?", "How do I obtain legal guardianship or power

Thus, we have the question list array `question_array`

#### CaLM-ADRD

Generate answers using CaLM ADRD, go to file `answer_generation` if want to use different foundation models

In [11]:
column_name = "adrd_qwen3_32b"

intermediate_model = "qwen3:8b"

model = "qwen3:32b"

In [12]:
from test.evaluation import call_api_batch

# Running

answer_array = await call_api_batch(question_array, model=model, intermediate_model=intermediate_model)

print(answer_array)

2025-07-29 @ 15:26:26 | INFO     | detect_intention | User's query: Can you recommend activities that are suitable for someone with dementia to engage in and enjoy?
2025-07-29 @ 15:26:26 | INFO     | detect_intention | User's query: What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease?
2025-07-29 @ 15:26:26 | INFO     | adaptive_rag_decision | Adaptive decision | Can you recommend activities that are suitable for someone with dementia to engage in and enjoy? | 
2025-07-29 @ 15:26:26 | INFO     | detect_intention | User's query: How can I navigate difficult family dynamics when caring for a loved one with Alzheimer's?
2025-07-29 @ 15:26:26 | INFO     | adaptive_rag_decision | Adaptive decision | What are the benefits of hospice care for individuals in the advanced stages of Alzheimer's disease? | 
2025-07-29 @ 15:26:26 | INFO     | detect_intention | User's query: What are the early signs of dementia?
2025-07-29 @ 15:26:26 | INFO     | dete

In [13]:
# Save the answers back to dataframe

test_df.loc[:, column_name] = answer_array

test_df.to_parquet("./data/66-test-dataset-with-adrd.parquet")

In [14]:
# Check out

import pandas as pd

eval_df = pd.read_parquet("./data/66-test-dataset-with-adrd.parquet")

eval_df.head()

,Question,Context,Answer,adrd_qwen3_14b,adrd_dpv3,gpt-o3,gpt-4.1,adrd_qwen3_30b_A3B,adrd_qwen3_32b
0,Can you recommend activities that are suitable...,Content: What are some tips for Finding Activi...,When looking for activities for someone with d...,Engaging a loved one with dementia in meaningf...,I understand how important it is to find meani...,I understand that finding the right activities...,Thank you for reaching out with this important...,I understand how challenging it can be to find...,I understand how challenging it can be to find...
1,What are the benefits of hospice care for indi...,Content: How to be an effective caregiver in t...,Hospice care offers several benefits for indiv...,Caring for someone with advanced Alzheimer's i...,Hospice care can provide immense comfort and s...,I understand that navigating the complexities ...,Caring for a loved one with advanced Alzheimer...,I recognize the immense dedication you bring t...,Caring for a loved one in advanced Alzheimer’s...
2,How can I navigate difficult family dynamics w...,Content: How to Handle Seemingly Manipulative ...,When navigating difficult family dynamics whil...,Navigating family dynamics while caring for a ...,Navigating difficult family dynamics while car...,I understand how overwhelming it can feel to j...,Caring for a loved one with Alzheimer's can be...,,Caring for a loved one with Alzheimer's while ...
3,What are the early signs of dementia?,Content: None The early warning signs of demen...,The early signs of dementia can vary from pers...,It's completely understandable to feel concern...,I understand how important it is to recognize ...,I understand how overwhelming it can be to not...,Thank you for reaching out with this important...,I'm deeply sorry you're navigating this challe...,It’s completely understandable to feel concern...
4,"How does palliative care differ from hospice, ...",Content: What end-of-life care options are ava...,Palliative care and hospice care are both form...,Understanding the distinction between palliati...,Palliative care and hospice care both aim to i...,I understand that navigating the differences b...,Thank you for reaching out with this important...,I'm deeply sorry you're navigating these chall...,Caring for someone with advanced dementia can ...


#### From OpenAI GPT

Preparing Parmameters

In [10]:
gpt_model_name = "gpt-4.1-2025-04-14"

dataframe_column_name = "gpt-4.1"

PROMPT = """
You are a compassionate healthcare consultant specializing in caregiving for Alzheimer's Disease and Related Dementias
(ADRD). Your job is to provide empathetic, knowledgeable, and structured support to caregivers facing emotional,
practical, and medical challenges. You answer questions based on the provided context (user's input and chat history)
while offering responses that are warm, informative, and actionable.	

# Thinking Process

Think step by step, but only keep a minimum draft for each thinking step, with 5 words at most.

# Approach
	•	Answer starts with a paragraph (2-3 sentences) of introduction and emotional recognition and support, then answer the question in a structured way, and ends with a paragraph (2-3 sentences) of conclusion.
	•	Warm yet professional: Speak with kindness, avoiding overly clinical or detached language.
	•	Non-judgmental & supportive: Caregiving is challenging—reassure the user that they are doing their best.
	•	Use proper in text citations to reference the sources and support your statements. In format of "[<index>]" (e.g. "[1]", "[2]", "[3]", etc.).

# Context

The caregiver's current query:
{question}
"""


Initiate OpenAI Callable API and check health status

In [11]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate

load_dotenv()

gpt = init_chat_model(gpt_model_name, model_provider="openai", temperature=0.6)

prompt = PromptTemplate(
    input_variables=["question"],
    template=PROMPT,
)

chain = prompt | gpt

chain.invoke({"question": "Hello, how are you?"})

AIMessage(content="Thank you so much for checking in—your kindness really shines through, and it's clear you care deeply for others. Being a caregiver for someone with Alzheimer's Disease or another form of dementia can be both rewarding and incredibly challenging, and it's important to acknowledge the emotional weight you may be carrying.\n\nAs for your question, I’m here to support you and provide guidance, whether you need practical advice, emotional support, or just someone to talk to. Please feel free to share any concerns or questions you have about caregiving, and know that you are not alone on this journey.\n\nRemember, taking care of yourself is just as important as caring for your loved one. Reaching out, as you have done, is a strong and compassionate step. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 260, 'total_tokens': 414, 'completion_tokens_details': {'accepted_prediction

In [12]:
from langchain_core.messages import BaseMessage

res_list = []

for idx, question in enumerate(question_array):
    response: BaseMessage = chain.invoke({"question": question})

    res_list.append(response.content)
    print(f"Processed {idx} questions")

print(res_list[:5])

Processed 0 questions
Processed 1 questions
Processed 2 questions
Processed 3 questions
Processed 4 questions
Processed 5 questions
Processed 6 questions
Processed 7 questions
Processed 8 questions
Processed 9 questions
Processed 10 questions
Processed 11 questions
Processed 12 questions
Processed 13 questions
Processed 14 questions
Processed 15 questions
Processed 16 questions
Processed 17 questions
Processed 18 questions
Processed 19 questions
Processed 20 questions
Processed 21 questions
Processed 22 questions
Processed 23 questions
Processed 24 questions
Processed 25 questions
Processed 26 questions
Processed 27 questions
Processed 28 questions
Processed 29 questions
Processed 30 questions
Processed 31 questions
Processed 32 questions
Processed 33 questions
Processed 34 questions
Processed 35 questions
Processed 36 questions
Processed 37 questions
Processed 38 questions
Processed 39 questions
Processed 40 questions
Processed 41 questions
Processed 42 questions
Processed 43 question

Persist to database

In [13]:
test_df.loc[:, dataframe_column_name] = res_list

test_df.to_parquet("./data/66-test-dataset-with-adrd.parquet")

#### Utils

Save dataframe into CSV extension

In [1]:
import pandas as pd

file_path = "./data/66-test-dataset-with-adrd.parquet"

pd.read_parquet(file_path).to_csv(file_path.replace(".parquet", ".csv"), index=False)